# Домашнее задание по лекции "Работа с текстом 2"

**Задание 1**: реализуйте задачу классификации на основе BERT-like модели и KNN на данных [Russian Intents Dataset с Kaggle](https://www.kaggle.com/datasets/constantinwerner/qa-intents-dataset-university-domain).

**Цель:** научиться создавать классификаторы текстов в условиях большого числа маленьких классов, состоящих из коротких текстов.

**Результат:** код для создания поискового векторного индекса + логика определения класса на основе близости к обучающим объектам (по ближайшему, по топ-N ближайших, и т. п.).

**Инструменты:** Python, приближённый KNN (nmslib/faiss/scann), модели из Hugging Face (Transformers).

In [ ]:
!pip install transformers
!pip install nmslib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 57.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.6.1-py2.py3-none-any.whl (188 kB)
  Created wheel for nmslib: filename=nmslib-2.1.1-cp310-cp310-linux_x86_64.whl size=13578646 sha256=2893d5f4f6ce2b2f3ebe4defd65da697e870b7229eb27561a75c95634462b036
  Stored in directory: /root/.cache/pip/wheels/21/1a/5d/4cc754a5b1a88405cad184b76f823897a63a8d19afcd4b9314
Successfully built nmslib


### импорт библиотек

In [ ]:
import numpy as np
import pandas as pd
import torch
import transformers

In [ ]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/paraphrase-xlm-r-multilingual-v1')
model = AutoModel.from_pretrained('sentence-transformers/paraphrase-xlm-r-multilingual-v1')

### загрузка данных

In [ ]:
df_train = pd.read_csv('/content/dataset_train.tsv', sep = '\t', names=['text', 'category'])
df_train

,text,category
0,мне нужна справка,statement_general
1,оформить справку,statement_general
2,взять справку,statement_general
3,справку как получить,statement_general
4,справку ммф где получаться,statement_general
...,...,...
13225,тупой,smalltalk_abuse
13226,робот бестолковый,smalltalk_abuse
13227,несообразительный,smalltalk_abuse
13228,ты бестолковый,smalltalk_abuse


In [ ]:
# LabelEncoder для категории
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(df_train.category)
label = le.transform(df_train.category)
df_train['label'] = label
df_train

,text,category,label
0,мне нужна справка,statement_general,114
1,оформить справку,statement_general,114
2,взять справку,statement_general,114
3,справку как получить,statement_general,114
4,справку ммф где получаться,statement_general,114
...,...,...,...
13225,тупой,smalltalk_abuse,103
13226,робот бестолковый,smalltalk_abuse,103
13227,несообразительный,smalltalk_abuse,103
13228,ты бестолковый,smalltalk_abuse,103


In [ ]:
len(df_train['label'].unique())

142

In [ ]:
# перемешивание train данных
n = len(df_train.index)
print(n)
shuffled_indices = np.random.permutation(n)
df_train = df_train.iloc[shuffled_indices]
df_train.head()

13230


,text,category,label
5732,стоимость интернета стоять.,nsunet_cost,81
1705,как мне заказать литература,loc_library,56
8587,19863 3 расписание на понедельник находится,sched_for_group_day,91
8687,пт расписание 200650 находится,sched_for_group_day,91
4251,дайте карту корпусов,campus_map,2


In [ ]:
# обрежем датасет, оставив 20% данных (так как из-за нехватки ресурсов падает сессия)
dftrain = df_train.iloc[:2600, ::]
dftrain

,text,category,label
5732,стоимость интернета стоять.,nsunet_cost,81
1705,как мне заказать литература,loc_library,56
8587,19863 3 расписание на понедельник находится,sched_for_group_day,91
8687,пт расписание 200650 находится,sched_for_group_day,91
4251,дайте карту корпусов,campus_map,2
...,...,...,...
4718,сколько студентов фия изучать,stat_numb_of_students,113
10746,корпус административный этаж второй столовка н...,loc_cafeteria_new_building_2_etage,28
4571,"Привет, дружище",smalltalk_greetings,106
68,получаться справка об обучении,conform,3


In [ ]:
len(dftrain.label.unique())

142

In [ ]:
df_test = pd.read_csv('/content/dataset_test.tsv', sep = '\t', names=['text', 'category'])
df_test

,text,category
0,как получить справку,statement_general
1,мне нужна справка,statement_general
2,справка студента эф петь,conform
3,справка студента фф оформлять,conform
4,как мне заказать справка об обучении,conform
...,...,...
878,как получить оплата дороги,travelfinaid_get
879,как получить финансовая поддержка поездки,travelfinaid_get
880,ты бесполезный,smalltalk_abuse
881,бот совершено глупый,smalltalk_abuse


In [ ]:
# обрежем датасет, оставив 20% данных (так как из-за нехватки ресурсов падает сессия)

dftest = df_test.iloc[:180, ::]
dftest

,text,category
0,как получить справку,statement_general
1,мне нужна справка,statement_general
2,справка студента эф петь,conform
3,справка студента фф оформлять,conform
4,как мне заказать справка об обучении,conform
...,...,...
175,как пройти институт автоматики и электрометрии...,loc_termophysinstitute
176,институт автоматики находится,loc_termophysinstitute
177,институт автоматики и электрометрии со ран идти,loc_termophysinstitute
178,институт автоматики разыскать,loc_termophysinstitute


### токенизация и создание эмбеддингов

In [ ]:
#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/paraphrase-xlm-r-multilingual-v1')
model = AutoModel.from_pretrained('sentence-transformers/paraphrase-xlm-r-multilingual-v1')

In [ ]:
train_sentences = dftrain.text.tolist()
test_sentences = dftest.text.tolist()

In [ ]:
encoded_input_train = tokenizer(train_sentences, padding=True, truncation=True, return_tensors='pt')
encoded_input_test = tokenizer(test_sentences, padding=True, truncation=True, return_tensors='pt')

In [ ]:
with torch.no_grad():
    train_emb = mean_pooling(model(**encoded_input_train),encoded_input_train['attention_mask'])
    test_emb = mean_pooling(model(**encoded_input_test),encoded_input_test['attention_mask'])

### Классификация  с помощью  nmslib

In [ ]:
import nmslib

index = nmslib.init(method='hnsw', space='cosinesimil')
index.addDataPointBatch(train_emb)
index.createIndex({'post': 2}, print_progress=True)

In [ ]:
# нахождение ближайшего соседа и запись результатов в массив

test_replik = []
knn_train_replik = []
test_label = []
model_label = []
distance = []


for n in dftest.index:
    id,dist=index.knnQuery(test_emb[n], k=1)
    test_replik.append(dftest.iloc[n].text)
    test_label.append(dftest.iloc[n].category)
    # print(n, dftest.iloc[n].text,dftest.iloc[n].category)

    for i,d in zip(id,dist):
        distance.append(d)
        knn_train_replik.append(dftrain.iloc[i].text)
        model_label.append(dftrain.iloc[i].category)
        # print(d,dftrain.iloc[i].text, dftrain.iloc[i].category)
    # print()


In [ ]:
# результирующая таблица
data_dict = {'test_replik' : test_replik, 'knn_train_replik' : knn_train_replik, 'distance' : distance, 'test_label' : test_label, 'model_label': model_label}
data = pd.DataFrame(data_dict)
data

,test_replik,knn_train_replik,distance,test_label,model_label
0,как получить справку,справка как получить,9.945691e-02,statement_general,statement_general
1,мне нужна справка,мне нужно отчислить,1.497867e-01,statement_general,status_kickout
2,справка студента эф петь,справка студента ггф петь,4.894137e-03,conform,conform
3,справка студента фф оформлять,справка студента ммф оформить,9.043515e-03,conform,conform
4,как мне заказать справка об обучении,как мне заказать справка студента,1.522868e-01,conform,conform
...,...,...,...,...,...
175,как пройти институт автоматики и электрометрии...,как пройти институт автоматики и электрометрии...,4.768372e-07,loc_termophysinstitute,loc_automationelectrometryinstitute
176,институт автоматики находится,институт автоматики находится,0.000000e+00,loc_termophysinstitute,loc_lawinstitute
177,институт автоматики и электрометрии со ран идти,институт автоматики и электрометрии со ран идти,0.000000e+00,loc_termophysinstitute,loc_automationelectrometryinstitute
178,институт автоматики разыскать,институт автоматики разыскать,0.000000e+00,loc_termophysinstitute,loc_philologyinstitute


In [ ]:
# поставим в поле accuracy = 1, там где найденная и истинная категория совпали, а 0, где не совпали
accuracy = (data['test_label'] == data['model_label'])
data.loc[accuracy, 'accuracy'] = 1
data['accuracy'] = data['accuracy'].fillna(0)

,test_replik,knn_train_replik,distance,test_label,model_label,accuracy
0,как получить справку,справка как получить,0.099457,statement_general,statement_general,1.0
1,мне нужна справка,мне нужно отчислить,0.149787,statement_general,status_kickout,0.0
2,справка студента эф петь,справка студента ггф петь,0.004894,conform,conform,1.0
3,справка студента фф оформлять,справка студента ммф оформить,0.009044,conform,conform,1.0
4,как мне заказать справка об обучении,как мне заказать справка студента,0.152287,conform,conform,1.0


In [ ]:
data.head(20)

,test_replik,knn_train_replik,distance,test_label,model_label,accuracy
0,как получить справку,справка как получить,0.099457,statement_general,statement_general,1.0
1,мне нужна справка,мне нужно отчислить,0.149787,statement_general,status_kickout,0.0
2,справка студента эф петь,справка студента ггф петь,0.004894,conform,conform,1.0
3,справка студента фф оформлять,справка студента ммф оформить,0.009044,conform,conform,1.0
4,как мне заказать справка об обучении,как мне заказать справка студента,0.152287,conform,conform,1.0
5,справка студента фит оформить,справка студента получать,0.044857,conform,conform,1.0
6,справка студента как мне заказать,справка студента фия как мне заказать,0.028248,conform,conform,1.0
7,справка об обучении получать,справка о обучении получаться,0.043634,conform,conform,1.0
8,справка студента ммф оформлять,справка студента ммф оформить,0.007632,conform,conform,1.0
9,справка о обучении как мне заказать,справка об обучении заказать,0.125950,conform,conform,1.0


In [ ]:
data.accuracy.value_counts()

1.0    120
0.0     60
Name: accuracy, dtype: int64

Вывод: ошибка 33%, точность модели 77%